# LangGraph Custom Tool 정의하기



In [2]:
%pip install langchain langchain_openai langsmith pandas langchain_experimental matplotlib langgraph langchain_core duckduckgo-search langchain-community chromadb


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 사용자 정의 함수

### @tool decorator

@tool 데코레이터는 사용자 정의 도구(custom tool)를 정의하는 가장 간단한 방법입니다. 
이 데코레이터는 기본적으로 함수 이름을 도구 이름으로 사용하지만, 첫 번째 인자로 문자열을 전달하여 이를 재정의할 수 있습니다.

또한, 함수의 닥스트링(docstring)을 도구의 설명으로 사용하므로 반드시 닥스트링을 제공해야 합니다.

In [9]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


@tool 데코레이터는 주석(annotation) parsing, 중첩 스키마(nested schemas), 기타 기능을 지원한다는 점에 유의하세요:

In [ ]:
from langchain_core.tools import tool
from typing import Annotated, List


@tool
def multiply_by_max(
    a: Annotated[int, "scale factor"],
    b: Annotated[List[int], "list of ints over which to take maximum"],
) -> int:
    """Multiply a by the maximum of b."""
    return a * max(b)


print(multiply_by_max.args_schema.model_json_schema())

In [10]:
@tool(parse_docstring=True)
def foo(bar: str, baz: int) -> str:
    """The foo.

    Args:
        bar: The bar.
        baz: The baz.
    """
    return bar


print(foo.args_schema.model_json_schema())

{'description': 'The foo.', 'properties': {'bar': {'description': 'The bar.', 'title': 'Bar', 'type': 'string'}, 'baz': {'description': 'The baz.', 'title': 'Baz', 'type': 'integer'}}, 'required': ['bar', 'baz'], 'title': 'foo', 'type': 'object'}


In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

In [11]:
tools = [multiply]
llm_with_tools = llm.bind_tools(tools)

In [12]:
from langchain import hub

# Fetches the latest version of this prompt
prompt = hub.pull("wfh/langsmith-agent-prompt:5d466cbc")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant.

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [13]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

runnable_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=runnable_agent, tools=tools, handle_parsing_errors=True
)

In [14]:
results = agent_executor.invoke({"input":"6*7을 계산해줘"})
results

{'input': '6*7을 계산해줘', 'output': '6 * 7의 결과는 42입니다.'}